### Game mit Eventloop
Wir benutzen einen Eventloop, um ein einfaches Spiel zu programmieren,
bei dem der Spieler einen Linienzug zeichnet. Die Linie setzt sich jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.
Kreuzt sich die Linie oder erreicht sie den Rand der Leinwand, endet das Spiel.

Wir denken uns die Linie als Schlange, die sich in einem NxN Gitter bewegt und immer länger wird.
Der Spielzustand ist in einem Dict gespeichert.
Die Position des Kopfes `(col, row)` ist dem Schlüssel `head` zugewiesen,
die von restlichen Körper besetzten Felder sind als
NxN Tabelle repräsentiert, die dem Schlüssel `body` zugewiesen ist
Besetzte Felder sind mit `True` markiert, leere mit `False`.
Die Spielkomponente hat eine Funktion `move(dx, dy)`, welche die aktuelle Kopfposition `(x, y)`,
auf `(x+dx, y+dy)` updated und `(x, y)` als besetzt markiert.

Der Spielzustand wird auf der Leinwand dargestellt. Dabei wird in der mitte der Feldes eine
Linie in Bewegungsrichtung gezeichnet.

Wird eine Pfeiltaste gedrückt, wird diese als erstes Element in eine Liste
`event_queue` eingefügt. 
Ist der Eventqueue  nicht leer, entfernt 
der Eventloop die zuletzt gedrückt Taste, andernfalls wird
die zuletzt gedrückt Taste beibehalten. Die Taste wird in der Variable `key` gespeichert.
Dann wird die Funktione `game.move` mit den Argumenten `key_move[key]` aufgerufen.

Der Event-Loop wird mit Hilfe der Funktion `threading.Timer`.
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.


```python
stop_event = threading.Event()
event_queue = []

key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }

def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()            
```


***
Die Spielkomponente unseres Spiels ist in `game.py` implementiert.  
Nachstehende testen wir diese, indem wir ein neues Spiel starten und dann
versuchen, ein Quadrat zweimal zu umrunden.  
`game.output` sagt uns, was vorgeht. Nach einer Runde kreuzt sich die Linie und
es ist Game-Over. 
***

In [ ]:
import game

# Right, Up, Left, Down -> Game over
moves = [(1, 0), (0, 1), (-1, 0), (0, -1)] * 2

game.new_game(nrow=5, ncol=5)
for dx, dy in moves:
    game.move(dx, dy)

game.state

***
Als nächstes testen wir den Eventloop.
Im Moment hängen wir die Pfeiltasten Tasten noch von Hand an den
Eventqueue an.

**Beachte**: Da das `stop_event` nicht gesetzt wird,
läuft der Thread weiter und verbraucht Ressourcen.
Der Thread muss von Hand gestoppt werden, durch `stop_event.set()`.
***

In [29]:
import threading
import game


stop_event = threading.Event()
event_queue = ['ArrowRight',  'ArrowUp', 'ArrowLeft', 'ArrowDown'] * 2


key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()

In [ ]:
event_queue.extend(['ArrowRight',  'ArrowUp', 'ArrowLeft', 'ArrowDown'] * 2)

game.new_game()
stop_event.clear()
event_loop()

In [ ]:
# laeuft der Eventloop Thread noch?
print(stop_event.is_set())
print('Running threads:', [thread.name for thread in threading.enumerate()[8:]])

# Eventloop stoppen
stop_event.set()

***
Fortsetzung im Notebook `Game_mit_Eventloop_2.ipynb`
***